# Multi Task Model

## Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from tensorflow.keras.applications import ResNet50

from loadData_alt import createDataset

## Model parameters

In [2]:
# Log parameters
model_name = 'mtm_classification_ds'
savedModelPath = f'../../log/saved_models/{model_name}'
tb_log_dir = f'../../log/tensorboard/{model_name}'
cp_filepath = f'../../log/cps/{model_name}/'

if not os.path.exists(cp_filepath):
    os.makedirs(cp_filepath)

cp_filepath += 'latest_weights.h5'

# Dynamic hyperparameters
learningRate = 0.001
doDataAugmentation = True
dropoutRate = 0.25
width_multiplier = 1
depth_multiplier = 1

# Training parameters
batch_size = 32
epochs = 10

## Model callbacks

In [3]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    # keras.callbacks.EarlyStopping(
    #                 monitor="val_loss",
    #                 patience=2,
    #                 verbose=1)
]

## Data parameters

In [4]:
image_height = 160
image_width = 160

use_age_groups = False
num_age_classes = 8 if use_age_groups else 122

age_classes_dict = {
    0 : "0-13",
    1 : "14-19",
    2 : "20-29",
    3 : "30-39",
    4 : "40-49",
    5 : "50-59",
    6 : "60+",
    7 : "-"
}

## Data Augmentation

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomBrightness(0.2),
    ]
)

## Creating the training datasets

In [6]:
training_ds, validation_ds = createDataset('../../data/m3/training', (image_height, image_width), batch_size, 0.2, use_age_groups=use_age_groups)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Load the ResNet model

In [7]:
# Loading either the MobileNet architecture model or the previously saved model, and freeze it for transfer learning
base = ResNet50(
                input_shape=(image_height, image_width, 3), # Optional shape tuple, only to be specified if include_top is False
                # alpha=width_multiplier, # Controls the width of the network. (Width multiplier)
                # depth_multiplier=depth_multiplier, # Depth multiplier for depthwise convolution. (Resolution multiplier)
                # dropout=dropoutRate, # Dropout rate. Default to 0.001.
                weights="imagenet",
                # input_tensor=None,
                # pooling='avg', # Optional pooling mode for feature extraction when include_top is False. (None, avg, max)
                include_top=False
                )
           
# Freeze the base model
base.trainable = False

inputs = keras.Input(shape=(image_height, image_width, 3))
# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
base_model = base(inputs, training=False)
base_model = keras.layers.GlobalAveragePooling2D()(base_model)
base_model = keras.layers.Dense(1024)(base_model)
base_model = keras.layers.Dropout(0.3)(base_model)

## Creating Task 1 (Face Detection) Top Model

In [8]:
# Add Dense layer
face_head = tf.keras.layers.Dense(256, activation='relu')(base_model)
face_head = keras.layers.BatchNormalization()(face_head)
face_head = keras.layers.Activation('relu')(face_head)
face_head = keras.layers.Dropout(0.2)(face_head)
face_head = tf.keras.layers.Dense(128, activation='relu')(face_head)
face_head = tf.keras.layers.Dense(64, activation='relu')(face_head)

# Final layer for binary classification
face_outputs = keras.layers.Dense(1, activation='sigmoid', name='face_output')(face_head)


## Creating Task 2 (Mask Detection) Top Model

In [9]:
# Add Dense layer
mask_head = tf.keras.layers.Dense(256, activation='relu')(base_model)
mask_head = keras.layers.BatchNormalization()(mask_head)
mask_head = keras.layers.Activation('relu')(mask_head)
mask_head = keras.layers.Dropout(0.2)(mask_head)
mask_head = tf.keras.layers.Dense(128, activation='relu')(mask_head)
mask_head = tf.keras.layers.Dense(64, activation='relu')(mask_head)

# Final layer for binary classification
mask_outputs = keras.layers.Dense(1, activation='sigmoid', name='mask_output')(mask_head)

## Creating Task 3 (Age Prediction) Top Model

In [10]:
# Add Dense layer
age_head = tf.keras.layers.Dense(256, activation='relu')(base_model)
age_head = keras.layers.BatchNormalization()(age_head)
age_head = keras.layers.Activation('relu')(age_head)
age_head = keras.layers.Dropout(0.2)(age_head)
age_head = tf.keras.layers.Dense(128, activation='relu')(age_head)
age_head = tf.keras.layers.Dense(64, activation='relu')(age_head)

# Final layer for binary classification
age_outputs = keras.layers.Dense(num_age_classes, activation='softmax', name='age_output')(age_head)

## Creating and compiling the final model

In [15]:

model = keras.Model(inputs, [face_outputs, mask_outputs, age_outputs])
keras.utils.plot_model(model, to_file=f'{model_name}.png', show_layer_activations=True)

# Using a joint loss function for the three tasks:
# [ Loss = gamma * Loss_task1 + gamma * Loss_task2 + gamma * Loss_task3 ]
# Because every task is dependant on every other task, the model receives the loss of every task when gamma > 0

gamma = 1

model.compile(
      optimizer=keras.optimizers.Adam(0.01), # Learning Rate?
            loss={
                  'face_output': 'binary_crossentropy', 
                  'mask_output': 'binary_crossentropy',
                  'age_output': 'categorical_crossentropy'
                  },
            loss_weights={
                  'face_output': 0.33 * gamma, 
                  'mask_output': 0.33 * gamma,
                  'age_output': 0.33 * gamma
                  }, 
            metrics={
                  'face_output': 'accuracy', 
                  'mask_output': 'accuracy',
                  'age_output': 'accuracy'
                  },
)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 5, 5, 2048)   23587712    ['input_5[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['resnet50[3][0]']               
 alAveragePooling2D)                                                                              
                                                                                            

## Training the model with the dataset

In [12]:
history = model.fit(
            training_ds,
            validation_data=validation_ds,
            epochs=epochs, 
            callbacks=callbacks,
            shuffle=True
        )

Epoch 1/10
353/353 [==============================] - ETA: 0s - loss: 0.7228 - face_output_loss: 0.0368 - mask_output_loss: 0.0365 - age_output_loss: 2.1171 - face_output_accuracy: 0.9890 - mask_output_accuracy: 0.9862 - age_output_accuracy: 0.4985
Epoch 1: saving model to ../../log/cps/mtm_classification_ds\latest_weights.h5
353/353 [==============================] - 989s 3s/step - loss: 0.7228 - face_output_loss: 0.0368 - mask_output_loss: 0.0365 - age_output_loss: 2.1171 - face_output_accuracy: 0.9890 - mask_output_accuracy: 0.9862 - age_output_accuracy: 0.4985 - val_loss: 2.6179 - val_face_output_loss: 0.1434 - val_mask_output_loss: 0.0466 - val_age_output_loss: 7.7430 - val_face_output_accuracy: 0.9727 - val_mask_output_accuracy: 0.9947 - val_age_output_accuracy: 0.4745
Epoch 2/10
353/353 [==============================] - ETA: 0s - loss: 0.6675 - face_output_loss: 0.0148 - mask_output_loss: 0.0125 - age_output_loss: 1.9955 - face_output_accuracy: 0.9963 - mask_output_accuracy: 0.

## Save the model

In [13]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../../log/saved_models/mtm_classification_ds\assets


INFO:tensorflow:Assets written to: ../../log/saved_models/mtm_classification_ds\assets
